# ISOM 3400 Project WebScraping
# Group 5

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as W
from selenium.webdriver.support import expected_conditions as E
from selenium.webdriver import ActionChains
import time
import openpyxl

In [7]:
#excel result saving path
path = 'REST_Analysis_JAPANESE2.xlsx'

# chromedriver excutable_path
executable_path = r'c:\Users\Chan Kin Yan\Desktop\ZOE\BUSI\ISOM\ISOM3400\project\chromedriver.exe'

# Helper Function for SAVING DATA 
- **Create excel file**
- **Helper function for loading excel file and appending data from info_list with info of the pages restaurants**

In [2]:
#create an excel file
wb = openpyxl.Workbook(); 
sheet = wb.active; sheet.title = "restaurant_info"
sheet.append(['name', 'price', 'bookmark', 'happy', 'sad', 'food_type','location',
                'num_branch', 'num_seat', 'pay_list', 'other_facility',
                'taste','decor','service','hygiene','value','overall'])
wb.save(path)

In [8]:
#function for sending data to the created workbook
def excel(info_list):
    wb = openpyxl.load_workbook(path) #load back the created excel file
    sheet = wb.active
    for i in info_list: #saving everypage data
        sheet.append(i)
    wb.save(path)  #save the file
    print("open and save info") 

# Helper Function for SORTING DATA
- **Check duplication**
- **Extract numbers from string**

In [9]:
#check duplication
def duplicate(name_list, name):
    if name in name_list:
        return True
    else: 
        name_list.append(name)
        return False

In [10]:
#get number from a string and then return a list 
def get_num(txt):
    return [s for s in txt if s.isdigit()]

# Funtion for SCRAPING DATA from web
- **Get data from particular Japanese restaurant**
- **Get data of multiples restaurants from searching result page** 

In [11]:
# get more info by clicking into the individual restaurant
def get_more_info(bs_more):
    time.sleep(2)
    #get rating
    try:
        rate_list = bs_more.find_all ('div',{'class':'header-score-details-right-item'})
        criteria = {}; total = 0
        for rate_item in rate_list:
            c_name = str(rate_item.find('div',{'class':'header-score-details-right-item-title'}).get_text()) 
            rate = rate_item.find_all('div')[1].attrs['class'][2]
            rate = int(get_num(rate)[0])
            total+=rate
            criteria[c_name]=rate
        criteria['overall']=total
    except:  criteria = 0
        
    # get number of branches
    try:
        num_branch=bs_more.find('section',class_='all-branches-section').a.get_text()
        num_branch=int(''.join(get_num(num_branch)))
    except:
        num_branch = 1

    #narrow down lable and location for following info scraping 
    rest_info = bs_more.find('div',class_='left-col').find('div',class_='or-section-group pois-filter-feature')
    
    # get payment method
    pay_list = [] #store payment method
    try:
        payment = rest_info.find('div',class_="comma-tags").find_all('span')
        for pay in payment:
            pay_list.append(str(pay.get_text()))
        pay_list=','.join(pay_list) # join to be a string for storing to excel
    except: pay_list = 0

    #get num_seat
    try:
        num_seat = str(rest_info.find('div',class_='content'))
        num_seat=int(''.join(get_num(num_seat)))
    except:  num_seat = 0

    # other facility
    try:
        other_info = rest_info.find_all('span',class_='condition-name')
        other_facility = [] # store the info of other facilities
        for item in other_info:
            other_facility+=item.contents
        other_facility = ','.join(other_facility)  # join to be a string for storing to excel
        if other_facility== None: other_facility = 0 #return 0 for indicating nothing for additional facilities
    except: other_facility = 0

    return list(criteria.values()), num_branch, pay_list, num_seat, other_facility


In [12]:
#get info of multiples restaurants from searching result page
def get_info(bs, district, info_list,url):
    global name_list
    item_list =  bs.find_all('section',class_='content-wrapper')
    info_jap=[]

    #get info of each restaurant in the searching result page
    for item in item_list:
        
        #get general info for each restuarant
        name = item.find('h2').a.string
        #check duplication
        if duplicate(name_list,name): continue
        print(name)

        try:
            price = item.find('div',{'class':'icon-info icon-info-food-price'}).span.string
        except AttributeError:
            continue
        
        try:
            bookmark = item.find('div',{'class':'text bookmarkedUserCount js-bookmark-count'}).get("data-count")
        except AttributeError:
            continue

        try:
            sad = item.find('span',{'class':'score highlight'}).string
        except AttributeError:
            continue
        
        try:
            happy = item.find('span',{'class':'score score-big highlight'}).string
        except AttributeError:
            continue
            
        try:
            food_type = item.find('li').string
        except AttributeError:
            continue

        try:
            location = item.find('div',{'class':'icon-info address'}).a.string
            if location != district: continue
        except AttributeError:
            continue

        #save data to a list
        one_res = [name, price, bookmark, happy, sad, food_type, location]
        
        # specially save info of janpanese restaurants for further searching 
        if 'Japan' in food_type:
            global jap; jap+=1
            info_jap.append(one_res)
        else:  # save the info of other types restaurant to the general info list
            info_list.append(one_res); time.sleep(2)
     
    
    #enter new page for extra info for particular Japanese restaurants
    m=0; print(len(info_jap))
    for i in range(len(info_jap)):
        name_japan = info_jap[i][0]
        m=i
        try:
            # click into the individual restaurant
            for i in range(5): # limit of tolerance of failing entering the restaurant page
                try:
                    #click into the page by finding its link using the name of that restaurant
                    moreinfo = W(driver,15).until(E.presence_of_element_located((By.PARTIAL_LINK_TEXT,str(name_japan))))
                    time.sleep(1)
                    moreinfo.click()
                except: # reload the searching result page to find the restaurant link again if any accident happened
                    driver.get(url); time.sleep(5); 
                    if i==4: driver.quit()
                    else: continue
                        
                try:
                    # using bs4 to get the info of the restaurant 
                    time.sleep(2)
                    bs_more=BeautifulSoup(driver.page_source, 'html.parser')
                    bs_more.prettify()
                    if bs_more: break
                except:  # reload from searching result page and redo the pervious poceedures
                    driver.get(url); time.sleep(5)
                    if i==4: driver.quit()
                
                
            #getting more information of the restaurant by calling functions
            try:  
                criteria, num_branch, pay_list, num_seat, other_facility = get_more_info(bs_more)    
                print('getting more')
            except: time.sleep(3) ; continue
                
            print('enter',name_japan)
            
            # go back to the searching result page to continue for other restaurants
            try:
                driver.execute_script("window.history.go(-1)")
                driver.implicitly_wait(1); time.sleep(3);  print('go_out')
            except:
                print('go back failed')
                driver.quit()
        except:
            print('cannot',name_japan); continue

        #save the additional data to the particular list of the restaurant 
        info_jap[m]+= [num_branch, num_seat, pay_list, other_facility]
        for j in criteria:
            info_jap[m].append(j)
        time.sleep(1)

    #sava info of japanese restaurant to info_list containing info in this page
    info_list+=info_jap
    time.sleep(2)

# Main function for navigating to pages and performing webscraping 

- **Launch Chrome driver**
- **Loop through district list to search result for each districts**
- **flipping pages results for each searching of the particular district**
- **Using functions perform webscraping to get information**

In [8]:
# Global variables for later used
url_main = 'https://www.openrice.com' # the major part of openrice website
url_district = ['Mong Kok','Causeway Bay','Central','Tsim Sha Tsui','Yuen Long','Tsuen Wan'] #list of district name we focus
jap = 0 # counting number of japanese restaurants in total
cookie_pressed = False # record whether cookie of the website got pressed
name_list = [] # name list of all the restaurants for checking duplication

In [9]:
#start chrome

#Hide automous control
options = webdriver.ChromeOptions() 
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')

# apply chromedriver
driver = webdriver.Chrome(options=options, executable_path=executable_path)
#driver = webdriver.Chrome() # replaced by the version which can hide automous control above

driver.maximize_window()

#Go to website of openrice
driver.get("https://www.openrice.com/en/hongkong/restaurants")

In [10]:
# search for every disticts in list

for district in url_district:
    #find search box
    try:
        search_box = driver.find_element_by_name("where")
        search_box.clear() 
        search_box.send_keys(district)
        search = driver.find_element_by_xpath('//*[@id="header"]/div[2]/div[4]/div/button').click()
        time.sleep(5)
    except:
        print('error'); driver.quit()

    #ready to get info
    endsearch = False #index for to break the loop when finishing searching the result of the district
    url= driver.current_url #get current url preventing from lost after clicking 

    #start to browse through every page of the searching results
    while not endsearch:
        info_list = []  #list for storing information from this page and later be stored into excel
        #cookies
        try:
            if (page == 1 and ( not cookie_pressed) ):
                print('cookie pressing')
                cookie = W(driver, 5).until(E.presence_of_element_located((By.XPATH,'//*[@id="cookies-agreement"]/div/button')))
                cookie.click()
                cookie_pressed = True
        except:
            print("no cookies")

        # get infomation in this page    
        time.sleep(5) #wait for most information present
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        bs.prettify()
        
        # call the webscraping function to perform 
        get_info(bs,district,info_list,url)

        #save data in this page
        excel(info_list)

        #go to next page
        #find the link of next page 
        time.sleep(3)
        next_page = bs.find('section', {'class':'js-pois-pagination pull-right'}).find('a',class_='pagination-button next js-next')      
        if next_page == None:
            endsearch = True
        else:
            next_page = next_page.attrs['href']
            url=url_main+str(next_page)
            driver.get(url) #go to next page
            time.sleep(10)
            
    print('done',district)

driver.quit()
print(jap) 

cookie pressing
Yadllie Plate
Tearapy 茶聊
Outdark (飛達商業中心)
Mini Friday (銀城廣場)
URBAN Cafe (彌敦道)
Sky726
Ocio (煙廠街)
Lockeroom Dining (朗晴坊)
Bellevue Bar and Grill
LAB EAT Restaurant & Bar (T.O.P This is Our Place)
HeSheEat (白布街)
Pyeong Chang 平昌
Station 3 Cafe 第三站
Double Cafe 孖啡
Aroy Khao Man Gai 阿來泰式地道風味
0
open and save info
E Pot (XX Arcade) 一鍋堂 (XX Arcade)
homemadecafe 部屋咖啡室
Running Chicken (The Lamma City) 米走雞 (太子藍馬之城)
Sawadee Thai 泰屋
Nagasakayaki 長阪燒
Mu Taiwan Noodles (Kwong Wa Street) 阿木台灣麵 (廣華街)
Poach 蟲二
Lazy Pot 吞POT 雞煲火鍋.住家菜
Japanese Delight 日本野
Running BBQ Running 燒
Kobekyu (Nathan Road) 吉列牛忌廉烏冬專門店 (彌敦道)
Shinko 申子居酒屋
Chiba 吃吧
Cheng Du Old Pier Hot Pot 成都老碼頭火鍋
Champak Restaurant (The Forest) 青花 (The Forest)
4
getting more
enter Nagasakayaki 長阪燒
go_out
getting more
enter Poach 蟲二
go_out
getting more
enter Japanese Delight 日本野
go_out
getting more
enter Shinko 申子居酒屋
go_out
open and save info
The Place
Hey Pot 喜鍋 海鮮‧火鍋‧小廚
二鍋頭 (福苑)
Go Dutch Cafe
811 Sichuan Cuisine 捌壹壹
OPPA Korean Restau

Tastory 美味故事
1
getting more
enter 燒烤堂
go_out
open and save info
富盈火鍋雞煲專門店
Ca-Tu-Ya 吉豚屋吉列豬扒專門店
La Maison D'Elephant 象屋泰國餐廳
Okonomiyaki Dohtonbori Restaurant (Langham Place) 道頓堀御好燒專門店 (朗豪坊)
Kingsley Cafe and Meal (T.O.P This is Our Place) 山下菓子 (T.O.P This is Our Place)
NMC Noodles (Soy Street) 農米莊 (豉油街)
Di King Heen 帝京軒
Mou Mou Club 牛涮鍋
Beijing Home 北京老家
大笪地花膠雞
Tea Only (Argyle Centre) 唇茶 (旺角中心)
Momiji Chaya (T.O.P This is Our Place) 紅葉茶屋 (T.O.P This is Our Place)
Coffee Green Tea 咖啡綠茶
一日三餐
The Sky Bar
5
getting more
enter Ca-Tu-Ya 吉豚屋吉列豬扒專門店
go_out
getting more
enter Okonomiyaki Dohtonbori Restaurant (Langham Place) 道頓堀御好燒專門店 (朗豪坊)
go_out
getting more
enter Kingsley Cafe and Meal (T.O.P This is Our Place) 山下菓子 (T.O.P This is Our Place)
go_out
getting more
enter Mou Mou Club 牛涮鍋
go_out
getting more
enter Momiji Chaya (T.O.P This is Our Place) 紅葉茶屋 (T.O.P This is Our Place)
go_out
open and save info
Mini Club
Xing Fu Tang 幸福堂
Shun by Kokoro Sushi 鮨心 ，旬節
Crazy Hot Pot 一鍋瘋
CR² By Cupping Ro

SUSHI YOROKOBU 鮨㐂
PLAYT PLAYT 咖啡廳
Sushi Itsu 鮨逸
Kyo Hayashiya (Hysan Place) 京林屋 (希慎廣場)
Milksha 迷客夏
Chicken Wings Mountain (Jaffe Road) 雞翼山 (謝斐道)
Yaki Oni 燒鬼
Kanada-ya 金田家
Fujisan 富士山
6
getting more
enter SUSHI YOROKOBU 鮨㐂
go_out
getting more
enter Sushi Itsu 鮨逸
go_out
getting more
enter Kyo Hayashiya (Hysan Place) 京林屋 (希慎廣場)
go_out
getting more
enter Yaki Oni 燒鬼
go_out
getting more
enter Kanada-ya 金田家
go_out
getting more
enter Fujisan 富士山
go_out
open and save info
Little Bao 小包包
On-Yasai 温野菜
The Leah
STEAK ONE 鉄板一
Mini Friday (銅鑼灣廣場2期)
N.O.T. Specialty Coffee
Rakuen (Circle Tower) 樂宴 (永光中心)
Al Forno Cafe and Bar
Wagyu 88 和牛88
Yan Wo Dou Bun Chong 人和荳品廠
Ming Bistro (Hysan Place) 名館 (希慎廣場)
Kyoku 旭
PUTIEN (Lee Theatre Plaza) 莆田 (利舞臺廣場)
Fung Lum 楓林小館
Marginal Gathering (Island Beverley) 緣味小聚 (金百利廣場)
4
getting more
enter On-Yasai 温野菜
go_out
getting more
enter Rakuen (Circle Tower) 樂宴 (永光中心)
go_out
getting more
enter Wagyu 88 和牛88
go_out
getting more
enter Kyoku 旭
go_out
open and save info
日

Ebisoba Ichigen 一幻拉麵
White Beard Fish n Chips 白鬍子炸魚薯條
ZEST by Konishi
Nijuuichi Don 廿一堂
Mr Wolf
Cupping Room Coffee Roasters (閣麟街)
Sushi Kuu 壽司喰
MeKong泰.越.港
Mandarin Grill + Bar 文華扒房及酒吧
Sakura Japanese Cuisine 櫻花日本料理
5
getting more
enter Ebisoba Ichigen 一幻拉麵
go_out
getting more
enter ZEST by Konishi
go_out
getting more
enter Nijuuichi Don 廿一堂
go_out
getting more
enter Sushi Kuu 壽司喰
go_out
getting more
enter Sakura Japanese Cuisine 櫻花日本料理
go_out
open and save info
Tonkichi Tonkatsu Seafood (IFC) 丼吉日式吉列豬排專門店 (國際金融中心)
Asian Cooking 薈亞廚房
Vive Cake Boutique
LPM Restaurant & Bar
Écriture
Golden Thai 金坊泰國美食
Green Common (將軍澳中心)
R&R Bagels
alfafa Cafe
The Chinese Banks' Association Ltd 香港華商銀行公會
Kind Kitchen
RUBIA
Hanagushi Yakitori Japanese Restaurant 花串日本料理
2
getting more
enter Tonkichi Tonkatsu Seafood (IFC) 丼吉日式吉列豬排專門店 (國際金融中心)
go_out
getting more
enter Hanagushi Yakitori Japanese Restaurant 花串日本料理
go_out
open and save info
The Roundhouse Taproom
Uma Nota
VEDA
Luk Yu Tea House 陸羽茶室
Sukiyaki

5
getting more
enter Grandmama Cafe 婆婆珈琲屋
go_out
getting more
enter Okonomiyaki Dohtonbori Restaurant (K11 Art Mall) 道頓堀御好燒專門店 (K11購物藝術館)
go_out
getting more
enter Gekiyasu Yakiniku 驚安燒肉商店
go_out
getting more
enter Sushi Yutaka 壽司豐
go_out
getting more
enter Izakaya - Honotorihane 炎鳥羽
go_out
open and save info
The Food Gallery
Empire City Roasted Duck 大都烤鴨
AFURI Ramen + Dumpling (The ONE) 阿夫利拉麵 (The ONE)
Kappo Ro Ann (iSQUARE) 割烹 櫓杏 (iSQUARE國際廣場)
Jan Jan Kushikatsu Jan Jan 串揚專門店
Greyhound Café Galleria (Victoria Dockside K11 Musea)
Lady Nara
Kyoto Ichinoden 京都一の傳
Dang Wen Li by Dominique Ansel 當文歷餅店 by Dominique Ansel
ANA TEN 天穴
東京食堂
The Blind Pig (諾士佛臺)
ÉPURE
Fu Rong (Ocean Terminal) 映水芙蓉 (海運大廈)
5
getting more
enter AFURI Ramen + Dumpling (The ONE) 阿夫利拉麵 (The ONE)
go_out
getting more
enter Kappo Ro Ann (iSQUARE) 割烹 櫓杏 (iSQUARE國際廣場)
go_out
getting more
enter Kyoto Ichinoden 京都一の傳
go_out
getting more
enter ANA TEN 天穴
go_out
getting more
enter 東京食堂
go_out
open and save info
sensory ZERO (

go_out
getting more
enter Gyujin Shabu Shabu & Sukiyaki Restaurant (Yoho Mall) 牛陣 (形點)
go_out
getting more
enter Miao sushi 淼ひよう壽司日本料理
go_out
getting more
enter Gyu-Kaku (Yuen Long Plaza) 牛角日本燒肉專門店 (元朗廣場)
go_out
open and save info
古月聚台灣古早味美食
Chicken Pot Hut 雞煲棧火鍋
蘇記流心燉蛋專門店
Tai Wing Wah Restaurant 大榮華酒樓
Sora Cafe
立哥麵家
Chow Chow Cafe
Moment Cafe
Spicy Drucken (Sky City) 醉辣鮮 (天空之城)
肥仔海南雞專門店
Brotzeit German Bier Bar & Restaurant (形點)
Sakura 桜
Ho To Tai Noodle Shop 好到底麵家
Prince Kitchen 王子私房菜
1
getting more
enter Sakura 桜
go_out
open and save info
The Cosmopolitan (Yuccie Square) 鍋心粥底火鍋 (安寧路)
辣媽川湘私房菜
勤記潮州食家
Brother 5 Bar and Restaurant
Pepper Lunch (形點)
Wagiku Teppanyaki 鐵板燒和菊
Thai Cow 泰Cow
梁記餃子麵坊
Sweet Canteen 大飯糖甜品小食屋
醋頭家
Yat Gor Dim Sum 一哥點心
魚眾不同酸菜魚 (紫荊樓)
Jade Garden (YOHO MALL) 翠園 (形點)
Lee Moon Palace 利滿皇宮
2
getting more
enter Pepper Lunch (形點)
go_out
getting more
enter Wagiku Teppanyaki 鐵板燒和菊
go_out
open and save info
熊本牛鍋 (世宙商場)
HABITŪ caffè (形點)
Thai Market
Taiwan Kitchen 台灣麵
三多
Motor

go_out
getting more
enter Maru Kyou 麵屋丸京
go_out
getting more
enter Totoya 峰壽司魚屋
go_out
getting more
enter Cow Bean (荃灣廣場)
go_out
getting more
enter ASAP As Simple As Possible
go_out
getting more
enter Book B Kitchen Mealmeal
go_out
open and save info
101 Taiwanese Restaurant (Kolour Tsuen Wan) 101台式休閒餐廳 (千色匯)
Ajisen Ramen (Discovery Park) 味千拉麵 (愉景新城)
贊和豆品
魚屋
Victoria Harbour Restaurant 海港酒家
Wagyu More 牛摩+
民豐粉麵行
Lao Cuo Bian  老厝邊
Ahimsa Buffet (Nina Tower) 無肉食 (如心廣場)
Big Bro Cafe 大兄冰室
Home Bar & Restaurant
3
getting more
enter Ajisen Ramen (Discovery Park) 味千拉麵 (愉景新城)
go_out
getting more
enter 魚屋
go_out
getting more
enter Wagyu More 牛摩+
go_out
open and save info
Dumpling Land 餃撚店
Teishoku 8 定食8
PizzaExpress (荃灣廣場)
道日本料理
Gyu Gyu Shabu Shabu 牛牛日式火鍋放題
Satay King (Chung On Street) 沙嗲王 (眾安街)
Casserole Buddies (Panda Place) 砂煲兄弟 (悅來坊)
Sinakawa Japanese Restaurant 品川日本料理
Browny Cafe 咖啡賞館
Bay's Kitchen & Bar
Sweet Basil Thai Cuisine 金不換泰國餐廳
Isari Kiya Japanese Dining (Citywalk) 漁屋日本料理 (荃新天地)
5
